In [44]:
from utils import *
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.decomposition import PCA

In [45]:
fd_collection = getCollection("team_5_mwdb_phase_2", "fd_collection")

In [46]:
selected_feature_model = "resnet_fd"

t = int(input("Enter value of t: "))
if t < 1:
    raise ValueError("t should be a positive integer")

num_layers = int(input("Enter the number of layers: "))
if num_layers < 1:
    raise ValueError("num_layers should be a positive integer")

num_hashes_per_layer = int(input("Enter the number of hashes per layer: "))
if num_hashes_per_layer < 1:
    raise ValueError("num_hashes_per_layer should be a positive integer")

ValueError: invalid literal for int() with base 10: ''

In [ ]:
all_images = list(fd_collection.find())
all_images = sorted(all_images, key = lambda x: x["image_id"])

odd_image_ids = [img["image_id"] for img in all_images if img["image_id"] % 2 == 0]

even_image_labels = [img["true_label"] for img in all_images if img["image_id"] % 2 == 0]
odd_image_labels = [img["true_label"] for img in all_images if img["image_id"] % 2 != 0]

feature_vectors = [np.array(img[selected_feature_model]).flatten() for img in all_images]

total_len = len(feature_vectors)
even_feature_vectors = []
odd_feature_vectors = []

for i in range(total_len):
  if i % 2 == 0:
    even_feature_vectors.append(feature_vectors[i])
  else:
    odd_feature_vectors.append(feature_vectors[i])

even_feature_vectors = np.array(even_feature_vectors)
odd_feature_vectors = np.array(odd_feature_vectors)

print(even_feature_vectors[0].shape)
print(odd_feature_vectors.shape)

odd_len = odd_feature_vectors.shape[0]
even_len = even_feature_vectors.shape[0]

(1000,)
(4338, 1000)


In [ ]:
query_image_id = int(input("Enter value of query_image_id: "))
if query_image_id < 1:
    raise ValueError("query_image_id should be a positive integer")

num_vectors = even_feature_vectors.shape[0]
vector_dimension = even_feature_vectors.shape[1]

lsh = LSHIndex(num_layers, num_hashes_per_layer, vector_dimension)

for index, vector in enumerate(even_feature_vectors):
    lsh.add_vector(vector.tolist(), index * 2)

query_vector = np.array(odd_feature_vectors[(query_image_id // 2)])

result_t_unique, unique_count, overall_count = lsh.query_t_unique(query_vector, t)
print(f"Number of unique vectors considered: {unique_count}")
print(f"Overall number of vectors considered: {overall_count}")
print(f"{t} Most Similar Vectors:")
print(f"Label of the input image: {odd_image_labels[(query_image_id // 2)]}")
for result in result_t_unique:
    print(f"Label: {even_image_labels[(result[0] // 2)]}, {result}")

Number of unique vectors considered: 3189
Overall number of vectors considered: 6922
15 Most Similar Vectors:
Label of the input image: 5
Label: 0, (0, [0.0010157952783629298, 0.0007815025164745748, 0.0006439489079639316, 0.0006443095626309514, 0.0008086328161880374, 0.00047308477223850787, 0.0004179290554020554, 0.0008153912494890392, 0.00044022544170729816, 0.0005768958362750709, 0.0007404272328130901, 0.000599000952206552, 0.000485737546114251, 0.000584599154535681, 0.0005976820248179138, 0.0003987941308878362, 0.0006087162182666361, 0.0006203220109455287, 0.00033957051346078515, 0.0005409272853285074, 0.000697399431373924, 0.0005742720095440745, 0.000626140448730439, 0.0006172480643726885, 0.0007455992745235562, 0.0005165034090168774, 0.0005326631362549961, 0.0007513134041801095, 0.0006662412197329104, 0.0005603042081929743, 0.000519087421707809, 0.0005670231766998768, 0.0005775297177024186, 0.0005526260938495398, 0.0002952576905954629, 0.0005033284542150795, 0.0007569431909359992,